In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d masoudnickparvar/brain-tumor-mri-dataset

 90% 133M/149M [00:00<00:00, 155MB/s]
100% 149M/149M [00:01<00:00, 154MB/s]


In [ ]:
import zipfile
zip_file_path = '/content/brain-tumor-mri-dataset.zip'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/brain_tumor_dataset')

In [ ]:
import numpy as np
import cv2
import os
import io
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import datetime
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from time import time
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard, LambdaCallback
from keras.layers import Input, Dropout, Dense, GlobalAveragePooling2D
from keras.models import Sequential, Model
from keras.applications.resnet import ResNet50


In [ ]:
# Define labels and image size
labels = ['glioma', 'meningioma', 'notumor', 'pituitary']
image_size = 150

# Function to preprocess images
def preprocess_image(image_path):
    image = cv2.imread(image_path, 0)
    image = cv2.bilateralFilter(image, 2, 50, 50)
    image = cv2.applyColorMap(image, cv2.COLORMAP_BONE)
    image = cv2.resize(image, (image_size, image_size))
    return image

# Load images and labels
x_train = []
y_train = []
x_test = []
y_test = []

for label in labels:
    trainPath = os.path.join('/content/brain_tumor_dataset/Training', label)
    for file in tqdm(os.listdir(trainPath)):
        image_path = os.path.join(trainPath, file)
        x_train.append(preprocess_image(image_path))
        y_train.append(labels.index(label))

    testPath = os.path.join('/content/brain_tumor_dataset/Testing', label)
    for file in tqdm(os.listdir(testPath)):
        image_path = os.path.join(testPath, file)
        x_test.append(preprocess_image(image_path))
        y_test.append(labels.index(label))

# Convert lists to NumPy arrays and normalize images
x_train = np.array(x_train) / 255.0
x_test = np.array(x_test) / 255.0
y_train = np.array(y_train)
y_test = np.array(y_test)

# Shuffle training data
x_train, y_train = shuffle(x_train, y_train, random_state=42)

# Perform one-hot encoding on labels
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

# Split dataset into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)



100%|██████████| 300/300 [00:01<00:00, 226.08it/s]


**Simple CNN**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(image_size, image_size, 3)),  # Adjust input shape for RGB images
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(labels), activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(),  # You can adjust the learning rate if needed
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
from time import time

# Train the model
start_time = time()
history = model.fit(x_train, y_train,
                    batch_size=32,
                    epochs=10,
                    validation_data=(x_val, y_val))
end_time = time()

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Print process time and metrics
print(f'Time taken: {end_time - start_time} seconds')
print(f'Training Acc: {np.mean(history.history["accuracy"])}')
print(f'Validation Acc: {np.mean(history.history["val_accuracy"])}')
print(f'Test Acc: {test_accuracy}')


Epoch 1/10
143/143 [==============================] - 260s 2s/step - loss: 0.8481 - accuracy: 0.6586 - val_loss: 0.5307 - val_accuracy: 0.8031
Epoch 2/10
143/143 [==============================] - 249s 2s/step - loss: 0.4652 - accuracy: 0.8295 - val_loss: 0.4043 - val_accuracy: 0.8443
Epoch 3/10
143/143 [==============================] - 253s 2s/step - loss: 0.3836 - accuracy: 0.8518 - val_loss: 0.3502 - val_accuracy: 0.8705
Epoch 4/10
143/143 [==============================] - 250s 2s/step - loss: 0.2933 - accuracy: 0.8949 - val_loss: 0.2843 - val_accuracy: 0.8933
Epoch 5/10
143/143 [==============================] - 250s 2s/step - loss: 0.2382 - accuracy: 0.9140 - val_loss: 0.2283 - val_accuracy: 0.9151
Epoch 6/10
143/143 [==============================] - 256s 2s/step - loss: 0.1800 - accuracy: 0.9348 - val_loss: 0.2257 - val_accuracy: 0.9195
Epoch 7/10
143/143 [==============================] - 251s 2s/step - loss: 0.1465 - accuracy: 0.9453 - val_loss: 0.1941 - val_accuracy: 0.9370

**Bayes by Backpropagation**

In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
import keras
import numpy as np

print("TensorFlow version:", tf.__version__)
print("TensorFlow Probability version:", tfp.__version__)
print("Keras version:", keras.__version__)
print("NumPy version:", np.__version__)


TensorFlow version: 2.15.0
TensorFlow Probability version: 0.23.0
Keras version: 2.15.0
NumPy version: 1.25.2


In [ ]:
from tensorflow.keras import layers, regularizers, models, optimizers
from time import time

# Define Bayesian Neural Network model with increased capacity and regularization
def build_bcnn_model(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(64, 3, activation="relu", kernel_regularizer=regularizers.l2(0.001))(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(128, 3, activation="relu", kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(256, 3, activation="relu", kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(4, activation="sigmoid")(x)  # Adjust output units to 4
    return models.Model(inputs, outputs)

# Instantiate the BCNN model
input_shape = (150, 150, 3)  # Assuming input shape is (150, 150, 3)
bcnn_model = build_bcnn_model(input_shape)

# Define and compile the Bayesian model
optimizer = optimizers.Adam(learning_rate=0.0001)  # Adjust learning rate
bcnn_model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])  # Use binary crossentropy for multi-label classification

# Train the BCNN model
start_time = time()
bcnn_history = bcnn_model.fit(x_train, y_train,
                              batch_size=32,
                              epochs=10,
                              validation_data=(x_val, y_val))
end_time = time()

# Evaluate the BCNN model on test data
test_loss, test_accuracy = bcnn_model.evaluate(x_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Print process time and metrics
print(f'Time taken: {end_time - start_time} seconds')
print(f'Training Acc: {bcnn_history.history["accuracy"][-1]}')  # Print final training accuracy
print(f'Validation Acc: {bcnn_history.history["val_accuracy"][-1]}')  # Print final validation accuracy
print(f'Test Acc: {test_accuracy}')


Epoch 1/10
143/143 [==============================] - 422s 3s/step - loss: 0.6409 - accuracy: 0.7649 - val_loss: 1.5751 - val_accuracy: 0.4436
Epoch 2/10
143/143 [==============================] - 426s 3s/step - loss: 0.4595 - accuracy: 0.8803 - val_loss: 1.0882 - val_accuracy: 0.5599
Epoch 3/10
143/143 [==============================] - 430s 3s/step - loss: 0.3911 - accuracy: 0.9230 - val_loss: 0.5424 - val_accuracy: 0.7664
Epoch 4/10
143/143 [==============================] - 430s 3s/step - loss: 0.3436 - accuracy: 0.9525 - val_loss: 0.4509 - val_accuracy: 0.8434
Epoch 5/10
143/143 [==============================] - 428s 3s/step - loss: 0.3227 - accuracy: 0.9610 - val_loss: 0.3831 - val_accuracy: 0.9081
Epoch 6/10
143/143 [==============================] - 427s 3s/step - loss: 0.2975 - accuracy: 0.9709 - val_loss: 0.3583 - val_accuracy: 0.9178
Epoch 7/10
143/143 [==============================] - 426s 3s/step - loss: 0.2718 - accuracy: 0.9858 - val_loss: 0.3726 - val_accuracy: 0.9081

**ResNet50**

In [ ]:
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping
from time import time

# Define ResNet50-based model
def build_resnet_model(input_shape):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    predictions = layers.Dense(4, activation='sigmoid')(x)  # Adjust output units to 4
    model = models.Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False  # Freeze pre-trained layers
    return model

# Instantiate the ResNet50-based model
input_shape = (150, 150, 3)  # Assuming input shape is (150, 150, 3)
resnet_model = build_resnet_model(input_shape)

# Compile the model
optimizer = optimizers.Adam(learning_rate=0.0001)  # Adjust learning rate if needed
resnet_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
start_time = time()
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)  # Add early stopping to prevent overfitting
resnet_history = resnet_model.fit(x_train, y_train,
                                  batch_size=32,
                                  epochs=10,  # Increase epochs if needed
                                  validation_data=(x_val, y_val),
                                  callbacks=[early_stopping])
end_time = time()

# Evaluate the model on test data
test_loss, test_accuracy = resnet_model.evaluate(x_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Print process time and metrics
print(f'Time taken: {end_time - start_time} seconds')
print(f'Training Acc: {resnet_history.history["accuracy"][-1]}')  # Print final training accuracy
print(f'Validation Acc: {resnet_history.history["val_accuracy"][-1]}')  # Print final validation accuracy
print(f'Test Acc: {test_accuracy}')


Epoch 1/10
143/143 [==============================] - 309s 2s/step - loss: 0.5409 - accuracy: 0.4338 - val_loss: 0.5026 - val_accuracy: 0.5022
Epoch 2/10
143/143 [==============================] - 308s 2s/step - loss: 0.4819 - accuracy: 0.5408 - val_loss: 0.4715 - val_accuracy: 0.5739
Epoch 3/10
143/143 [==============================] - 303s 2s/step - loss: 0.4570 - accuracy: 0.5892 - val_loss: 0.4537 - val_accuracy: 0.6308
Epoch 4/10
143/143 [==============================] - 313s 2s/step - loss: 0.4400 - accuracy: 0.6225 - val_loss: 0.4414 - val_accuracy: 0.6045
Epoch 5/10
143/143 [==============================] - 328s 2s/step - loss: 0.4272 - accuracy: 0.6273 - val_loss: 0.4289 - val_accuracy: 0.6457
Epoch 6/10
143/143 [==============================] - 307s 2s/step - loss: 0.4145 - accuracy: 0.6500 - val_loss: 0.4171 - val_accuracy: 0.6273
Epoch 7/10
143/143 [==============================] - 328s 2s/step - loss: 0.4032 - accuracy: 0.6614 - val_loss: 0.4044 - val_accuracy: 0.6597